# Load(url), save to file

In [1]:
# import urllib.request
# url = 'http://cirtec.ranepa.ru/Word2Vec/fixes.raw.txt'
# response = urllib.request.urlopen(url)
# data = response.read()      # a `bytes` object

In [2]:
# with open('./Word2Vec/fixes.raw.txt', 'wb') as f:
#     f.write(data)

# load file

In [2]:

with open('./Word2Vec/fixes.raw.txt', 'rb') as f:
    data = f.read()

# Get data

In [3]:
text = data.decode('utf-8') # a `str`; this step can't be used if data is binary

In [3]:
import string
import re

# regex = re.compile('[%s]' % re.escape(string.punctuation))
regex = re.compile('[^а-яА-Я]')

In [5]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [6]:
def get_average_len_words(sentence):
    words = sentence.split()
    return sum([len(word) for word in words]) / len(words)

In [8]:
# 10,000 ~ 3 минуты

In [1]:
%%time
import pandas as pd

count_of_sentences = 100000

df_lines = pd.DataFrame([line.split(" ", 4) for line in text.split("\n")][:count_of_sentences],
                        columns= ["citing_1", "citing_2", "citing_3", "start_ind", "text"])
df_lines = df_lines[~df_lines.text.isna()]
df_lines["average_len"] = df_lines.text.apply(get_average_len_words)
df_lines = df_lines[df_lines.average_len >= 4]
df_lines["normal_words"] = df_lines.text.apply(lambda text: [morph.parse(word)[0].normal_form for word in regex.sub(' ', text).split()])
df_lines.shape

CPU times: user 422 ms, sys: 438 ms, total: 859 ms
Wall time: 365 ms


In [14]:
df_lines.head(4)

,citing_1,citing_2,citing_3,start_ind,text,average_len,normal_words
0,citing:spz:neicon,citing:neicon:niidi,citing:y:2017:i:2:p:13-16,start_2027,"Key words: intoxication, glutathione S-transfe...",7.625000,"[ми, туберкулёз, лёгкое, выполнить, фармакоген..."
1,citing:spz:neicon,citing:neicon:niidi,citing:y:2017:i:2:p:13-16,start_2521,Среди них наибольшее значение в метаболизме кс...,6.797101,"[среди, они, наибольший, значение, в, метаболи..."
2,citing:spz:neicon,citing:neicon:niidi,citing:y:2017:i:2:p:13-16,start_2811,Анализ данных литературы свидетельствует о нал...,7.876923,"[анализ, дать, литература, свидетельствовать, ..."
3,citing:spz:neicon,citing:neicon:niidi,citing:y:2017:i:2:p:13-16,start_3115,Физическое исследование больного туберкулёзом ...,7.906977,"[физический, исследование, больной, туберкул, ..."


In [15]:
from gensim.models.phrases import Phrases

In [16]:
bigram = Phrases(df_lines.normal_words.values.tolist(), min_count=2, threshold=10)
trigram
df_lines["map_normal_words"] = [bigram[normal_words] for normal_words in df_lines.normal_words]

/usr/local/lib/python3.5/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [17]:
sorted_by_value = sorted(dict(bigram.vocab).items(), key=lambda kv: kv[1], reverse=True)

In [21]:
# import nltk
# nltk.download()

from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
stop_words = set(stopwords.words('russian'))

sorted_by_value_not_stop_words = [value for value in sorted_by_value if value[0].decode("utf-8") not in stop_words]

In [23]:
for word, count in sorted_by_value_not_stop_words[:100]:
    print(word.decode("utf-8"))

исследование
год
который
тип
являться
больной
дать
развитие
ребёнок
это
случай
желудок
соавт
и_соавт
рак
снижение
уровень
опухоль
также
мочь
лечение
пациент
время
ранний
фактор
тромбоз
иметь
стопа
высокий
результат
хемерин
рак_желудок
у_ребёнок
наличие
изменение
заболевание
весь
автор
группа
число
терапия
ррж
инвазия
костя
клетка
афл
однако
сустав
показать
система
соавт_и
лерканидипин
работа
метод
кровь
применение
процесс
прогноз
хрящ
активность
и_в
качество
клинический
в_год
анализ
ребёнок_с
образование
костный
препарат
ткань
частота
классификация
структура
влияние
течение
у_больной
коллаген
риска
мутация
ба
цель
доза
отметить
коллаген_тип
ра
а_также
повышение
аденокарцинома
ранний_рак
наиболее
риск
оболочка
эффект
технология
степень
различный
использование
связь
важный
согласно
